In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import os


# Pergunta pro sistema qual a pasta atual
PastaAtual = os.getcwd()

#cria uma variavel contendo o endereço da pasta com os csv
PastaCsvs= PastaAtual+"/Arquivos CSV"


#sistema, me liste por favor, os arquivos dentro desta pasta:
Arquivos = os.listdir(PastaCsvs) 

# Função para Listar Items
def Lista_item ():

    for i, elemento in enumerate(Arquivos):
        print (f"[{i}] - {elemento}")

# Função para a leitura das tabelas:

def Seleciona_automatico(nome:str):
    try:
        for Arquivo in Arquivos:
            if Arquivo == nome:
                return pd.read_csv(PastaCsvs+"/"+Arquivo, sep=",")
    except ValueError:
        print("O Arquivo nao foi encontrado")
        
# Aqui vamos mostrar os arquivos

Lista_item()

# Aqui lemos para a memoria todos os arquivos e armazenamos em uma variavel chamada Data
Data = {}
for Arquivo in Arquivos:
    Nome = Arquivo.replace(".csv","")
    Data[Nome]=Seleciona_automatico(Arquivo)

[0] - J2-BGA_S1_1.csv
[1] - S1(2)_2J1J2.csv
[2] - J2-BGA_S1_2.csv
[3] - Keteck_Samtec.csv
[4] - ConectorC_1.csv
[5] - J1-BGA_S1_1.csv
[6] - ConectorC_2.csv
[7] - J1-BGA_S1_2.csv
[8] - S1(2)_1J1J2.csv


In [2]:
# esta função processa os dados e retira os valores de NC alterando-os por 0
def TiraNC(J):
    J_ = deepcopy(J)
    for index, i in enumerate(J_):
        if i=="NC":
            J_[index]= "0"
        if i.isdigit():
            if int(i)<10:
                J_[index]=f"0{i}"
    return J_



#Função correlaciona os valores de JX com os valores de cada um dos conectores (C1_C2)
def JX_BGA ( J_X_BGA ,ConectorX,Num):
    
    J1_ = deepcopy(np.asarray(J_X_BGA[f'J{Num}']))
    BGA_J = deepcopy(np.asarray(J_X_BGA['BGA']))
    
    SaidaJ = []
    for alvo in ConectorX[f"J{Num}"]:
        if alvo !="0":
            #print(alvo,BGA_J_1[int(alvo)-1])
            
            SaidaJ.append(BGA_J[int(alvo)-1])
        else:
            SaidaJ.append("0")  
    
    SaidaJ_1=np.asarray(SaidaJ)
    Conector= deepcopy(ConectorX)
    Conector["BGA"]=SaidaJ
    return Conector
    
    

#Correlaciona o valor de J1 com o respectivo valor de BGA  
def Pixel_channel(CJX_1,CJX_2, Num):
    
    J1=np.asarray(deepcopy(CJX_1[f"J{Num}"]))
    J1_BGA=np.asarray(deepcopy(CJX_1["BGA"]))
    J1_Pixel=np.asarray(deepcopy(CJX_1["SiPM Pixel"]))
    
    J2 =np.asarray(deepcopy(CJX_2[f"J{Num}"]))
    J2_BGA=np.asarray(deepcopy(CJX_2["BGA"]))
    J2_Pixel=np.asarray(deepcopy(CJX_2["SiPM Pixel"]))

    
    J_Geral= []
    BGA_Geral = []
    Pixel_Geral = []
    
    if ((J1.shape == J1_BGA.shape) == (J2.shape == J2_BGA.shape))==(J1_Pixel.shape == J2_Pixel.shape):
        length =  J1.shape[0]
        for i in range (length):
            J_Geral.append(J1[i])
            BGA_Geral.append(J1_BGA[i])
            Pixel_Geral.append(J1_Pixel[i])
        
        for i in range (length):
            J_Geral.append(J2[i])
            BGA_Geral.append(J2_BGA[i])
            Pixel_Geral.append(J2_Pixel[i])
    
    ChannelBool= []
    OrderedChan = []
    
    for item in BGA_Geral:
        
        if item.find("CH")!=-1:
            ChannelBool.append(True)
            aux=int(item.replace("CH",""))
            if (aux < 10) :
                    OrderedChan.append(f"CH0{aux}")
            else:
                    OrderedChan.append(item)
                    
        else:
            ChannelBool.append(False)
            OrderedChan.append("NaN")
  
    
    OrgData ={}
    OrgData["J_Geral"]=J_Geral
    OrgData["BGA_Geral"]=BGA_Geral
    OrgData["Pixel"]=Pixel_Geral
    OrgData["Ch_bool"]=ChannelBool
    OrgData["Channel"]=OrderedChan
    OrgData= pd.DataFrame(OrgData)
    
    return OrgData

#calcula os valores de X e Y em base ao pixel detectado
def CalculateXY(Channels):
    
    Channels = deepcopy(Channels)
    Y = {
            'A' : 1,
            'B' : 2,
            'C' : 3,
            'D' : 4,
            'E' : 5,
            'F' : 6,
            'G' : 7,
            'H' : 8
    
    }
    x = []
    y = []
    for channel in Channels["Pixel"]:
        y.append(Y[channel[0]])
        x.append(channel[1]) 
    
    Channels["X"]=x
    Channels["Y"]=y

    return Channels


#Devolve Matriz Corrigida com valores de X e Y e salva a matriz resultante em HDD
def ResultadoFinal (Correlacao_1,Correlacao_2):
    Correlacao_1 = deepcopy(Correlacao_1)
    Correlacao_2 = deepcopy(Correlacao_2)
    
    Canais_1 = Correlacao_1[Correlacao_1["Ch_bool"]]
    Canais_2 = Correlacao_2[Correlacao_2["Ch_bool"]]
    
    
    SaveBoth = True
    if Canais_1.equals(Canais_2):
        print("J1 e J2 São Iguais")
        SaveBoth = False;
    else :
        print("Os dois DataFrames são Distintos")
        
    if not SaveBoth:
        Canais_1 = CalculateXY(Canais_1)
        Out = Canais_1.sort_values("Pixel")
        print("Salvando arquivo : PixelCorrigido.csv")
        Canais_1.to_csv("PixelCorrigido.csv")
        return Canais_1
    else :
        Canais_1 = CalculateXY(Canais_1)
        Out = Canais_1.sort_values("Pixel")
        print("Salvando arquivo : PixelCorrigido_J1.csv")
        Canais_1.to_csv("PixelCorrigido_J1.csv")
        Canais_2 = CalculateXY(Canais_2)
        Out_2 = Canais_2.sort_values("Pixel")
        Canais_1.to_csv("PixelCorrigido_J2.csv")
        print("Salvando arquivo : PixelCorrigido_J2.csv")
        return Canais_1, Canais_2
        

In [4]:
# Primeiro passo transformar A pinagem dos conectores C1 e C2 nas pinagens Lidas pelo socket Samtek

Conector1=Data["ConectorC_1"]
Conector2=Data["ConectorC_2"]

Ketek_Samtec=Data["Keteck_Samtec"]

#ordernar os dados de modo a que os canais da SiPM estejam organizados em ordem crescente
Aux =Ketek_Samtec.sort_values("HPK/KETEK Pin")

#junta as informações da pinagem Samtec aos Pixeis Ketek
Conector1["Samtec"]=np.asarray(Aux["SAMTEC Pin"])
Conector1=deepcopy(Conector1.sort_values("Samtec"))

#Processa os dados para encontrar Canal associado ao conector1
S_1_J1J2=Data["S1(2)_1J1J2"]
J1_1 = TiraNC(np.asarray(S_1_J1J2["J1"]))
J2_1 = TiraNC(np.asarray(S_1_J1J2["J2"]))
Conector1_J1 = deepcopy(Conector1)
Conector1_J2 = deepcopy(Conector1)

#organiza os dados do conector 1 em relação a J1
Conector1_J1["J1"]=J1_1
J1_BGA_S1_C1 = Data['J1-BGA_S1_1']
BGA_C1_J1 = JX_BGA(J1_BGA_S1_C1,Conector1_J1,1)
                   
#organiza os dados do conector 1 em relação a J2
Conector1_J2["J2"]=J2_1
J2_BGA_S1_C1 = Data['J2-BGA_S1_1']
BGA_C1_J2 = JX_BGA(J2_BGA_S1_C1,Conector1_J2,2)

#Processa os dados para encontrar Canal associado o conector 2

Conector2["Samtec"]=np.asarray(Aux["SAMTEC Pin"])
Conector2=deepcopy(Conector2.sort_values("Samtec"))
S_2_J1J2=Data["S1(2)_2J1J2"]
J1_2 = TiraNC(np.asarray(S_2_J1J2["J1"]))
J2_2 = TiraNC(np.asarray(S_2_J1J2["J2"]))
Conector2_J1 = deepcopy(Conector2)
Conector2_J2 = deepcopy(Conector2)


#organiza os dados do conector 2 em relação a J1
Conector2_J1["J1"]=J1_2
J1_BGA_S1_C2 = Data['J1-BGA_S1_2']
BGA_C2_J1 = JX_BGA(J1_BGA_S1_C2,Conector2_J1,1)

#organiza os dados do conector 2 em relação a J2
Conector2_J2["J2"]=J2_2
J2_BGA_S1_C2 = Data['J2-BGA_S1_2']
BGA_C2_J2 = JX_BGA(J2_BGA_S1_C2,Conector2_J2,2)


""" Texto meramente para organização

Variaveis de saida :

BGA_C1_J1 = JX_BGA(J1_BGA_S1_C1,Conector1_J1,1)
BGA_C1_J2 = JX_BGA(J2_BGA_S1_C1,Conector1_J2,2)
BGA_C2_J1 = JX_BGA(J1_BGA_S1_C2,Conector2_J1,1)
BGA_C2_J2 = JX_BGA(J2_BGA_S1_C2,Conector2_J2,2)

"""

#Canais associados ao conector J1 e J2
J1_Chan = Pixel_channel(BGA_C1_J1,BGA_C2_J1, 1)
J2_Chan = Pixel_channel(BGA_C1_J2,BGA_C2_J2, 2)


#Resultado Final
Tabela_Final = ResultadoFinal(Pixel_channel(BGA_C1_J1,BGA_C2_J1, 1),Pixel_channel(BGA_C1_J2,BGA_C2_J2, 2))




J1 e J2 São Iguais
Salvando arquivo : PixelCorrigido.csv
